# Trabajo Final Analisis Predictivo Javier Gilabert Sabater

# Carga y Limpieza de Datos

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd

#obtain hierarchical dataset
from datasetsforecast.hierarchical import HierarchicalData

# compute base forecast no coherent
from statsforecast.core import StatsForecast
from statsforecast.models import AutoARIMA, Naive

In [3]:
### Leer documento
df = pd.read_csv("H2.csv", sep = ",")

In [4]:
df.ReservationStatus.unique()

array(['Check-Out', 'Canceled', 'No-Show'], dtype=object)

In [5]:
df.IsCanceled.unique()

array([0, 1])

In [6]:
# Vemos como los valores "No-Show" y "Canceled" son clasificados como 1, "Check-out" esta clasificado como 0

In [7]:
import pandas as pd

# Supongamos que tu DataFrame se llama df y la columna de meses es 'Month'
# Creamos un diccionario de mapeo de nombres de meses a números
month_mapping = {
    'January': 1,
    'February': 2,
    'March': 3,
    'April': 4,
    'May': 5,
    'June': 6,
    'July': 7,
    'August': 8,
    'September': 9,
    'October': 10,
    'November': 11,
    'December': 12
}

# Aplicamos el mapeo a la columna 'Month' para convertir los nombres de meses a números
df['mes_numerico'] = df['ArrivalDateMonth'].map(month_mapping)

In [8]:
import pandas as pd

# Assuming your DataFrame is named df
# Create a new column 'ArrivalDate' by combining the year, month, week number, and day of month
df['ArrivalDate'] = pd.to_datetime(df['ArrivalDateYear'].astype(str) + '-' +
                                    df['mes_numerico'].astype(str) + '-' +
                                    df['ArrivalDateDayOfMonth'].astype(str))

# Drop the individual columns if needed
df = df.drop(['ArrivalDateYear', 'ArrivalDateMonth', 'ArrivalDateWeekNumber', 'ArrivalDateDayOfMonth', 'mes_numerico'], axis=1)


In [9]:
df = df.set_index('ArrivalDate')
df.head()

,IsCanceled,LeadTime,StaysInWeekendNights,StaysInWeekNights,Adults,Children,Babies,Meal,Country,MarketSegment,...,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,ReservationStatusDate
ArrivalDate,,,,,,,,,,,,,,,,,,,,,
2015-07-01,0,6,0,2,1,0.0,0,HB,PRT,Offline TA/TO,...,No Deposit,6,NULL,0,Transient,0.0,0,0,Check-Out,2015-07-03
2015-07-01,1,88,0,4,2,0.0,0,BB,PRT,Online TA,...,No Deposit,9,NULL,0,Transient,76.5,0,1,Canceled,2015-07-01
2015-07-01,1,65,0,4,1,0.0,0,BB,PRT,Online TA,...,No Deposit,9,NULL,0,Transient,68.0,0,1,Canceled,2015-04-30
2015-07-01,1,92,2,4,2,0.0,0,BB,PRT,Online TA,...,No Deposit,9,NULL,0,Transient,76.5,0,2,Canceled,2015-06-23
2015-07-02,1,100,0,2,2,0.0,0,BB,PRT,Online TA,...,No Deposit,9,NULL,0,Transient,76.5,0,1,Canceled,2015-04-02


In [10]:
df_status = df.iloc[:, :1]
df_status

,IsCanceled
ArrivalDate,
2015-07-01,0
2015-07-01,1
2015-07-01,1
2015-07-01,1
2015-07-02,1
...,...
2017-08-30,0
2017-08-31,0
2017-08-31,0


In [16]:
# Hacer un resample con periodos semanales
df_w_r = df_status.resample("W").count()
df_w_r.tail()

,IsCanceled
ArrivalDate,
2017-08-06,703
2017-08-13,717
2017-08-20,797
2017-08-27,678
2017-09-03,349


In [17]:
df_w_c = df_status.resample("W").sum()
df_w_c.tail()

,IsCanceled
ArrivalDate,
2017-08-06,251
2017-08-13,275
2017-08-20,332
2017-08-27,206
2017-09-03,101


------